In [74]:
import json
import numpy as np
import pandas as pd

In [75]:
raw_data = pd.read_csv('SAStimbreTags.csv', index_col=None, header=0)

with open('file_mapping.json', 'r') as file:
    file_mapping = json.load(file)

In [76]:
clean_data = {}
num_songs = 128
for i in range(num_songs):
    song_id = i + 3
    clean_data[song_id] = {'Safety': list(raw_data.iloc[:, 2 * i + 1]),
                           'Urgency': list(raw_data.iloc[:, 2 * i + 2]),
                           'Name' : file_mapping[i+2][1].split("/")[1].rstrip(".wav")}

json_object = json.dumps(clean_data, indent=4)
 

with open("TimbreResults.json", "w") as outfile:
    outfile.write(json_object)

In [77]:
id_to_filepath = {}
for file_pair in file_mapping:
    id = file_pair[0].split("/")[-1]
    id_to_filepath[id] = file_pair[1]

In [79]:
with open('TimbreResults.json', 'r') as file:
    timbre_results = json.load(file)

filepath = []
avg_safety = []
avg_urgency = []
for key in timbre_results.keys():
    if key + ".wav" in id_to_filepath.keys():
        safety_result = timbre_results[key]["Safety"]
        urgency_result = timbre_results[key]["Urgency"]
        if safety_result and urgency_result:
            filepath.append(id_to_filepath[key+".wav"])
            avg_safety.append(np.mean(safety_result).item())
            avg_urgency.append(np.mean(urgency_result).item())

df = pd.DataFrame({"filepath": filepath,
              "average safety": avg_safety,
              "average urgency": avg_urgency})

df.to_csv("average_ratings.csv")